<h1>Crypto Market Simulator</h1>

In [15]:
import asyncio
import ujson
import time
import sys 
import pandas as pd

from gate_ws import Configuration, Connection, WebSocketResponse
from gate_ws.spot import SpotPublicTradeChannel, SpotOrderBookChannel, SpotOrderBookUpdateChannel, SpotBookTickerChannel

In [16]:
NUM_OF_SNAPSHOTS = 5
curr_snapshot = 0
orders = []

<h2>Function to get orderbook data in json format</h2>

In [17]:
def print_message(conn: Connection, response: WebSocketResponse):
    global curr_snapshot
    global NUM_OF_SNAPSHOTS
    global orders
    if response.error:
        print('error returned: ', response.error)
        conn.close()
        return
    
    data = response.result
    if list(data.keys())[0] != "status":
        orders.append(data)
        curr_snapshot += 1
    
    if curr_snapshot >= NUM_OF_SNAPSHOTS:
        conn.close()
        return
        
async def main():
    channel_name = "BTC_USD"
    depth = "20"
    update_rate = "1000ms"
    
    filename = "orderbook.json"
    
    conn = Connection(Configuration())
    
    channel = SpotOrderBookChannel(conn, print_message)
    
    channel.subscribe([channel_name, depth, update_rate])

    await conn.run()
    
    with open(filename, "w") as file:
        data = ujson.dumps(orders, indent=4)
        print(data)
        file.write(data)

if __name__ == '__main__':
    start_time = time.perf_counter()
    await main()
    end_time = time.perf_counter()
    print(f"Time: {end_time - start_time}")

[
    {
        "t": 1671462567378,
        "lastUpdateId": 867920009,
        "s": "BTC_USD",
        "bids": [
            [
                "16695.22",
                "0.0063"
            ],
            [
                "16695.21",
                "0.025"
            ],
            [
                "16695.2",
                "0.2883"
            ],
            [
                "16689.34",
                "0.0484"
            ],
            [
                "16686.8",
                "0.551"
            ],
            [
                "16684.71",
                "0.0035"
            ],
            [
                "16677.91",
                "0.5404"
            ],
            [
                "16676",
                "0.0244"
            ],
            [
                "16669.48",
                "0.0063"
            ],
            [
                "16662.67",
                "0.0257"
            ],
            [
                "16656.12",
                "0.0024"
       

In [18]:
df = pd.read_json("orderbook.json")

df

,t,lastUpdateId,s,bids,asks
0,1671462567378,867920009,BTC_USD,"[[16695.22, 0.0063], [16695.21, 0.025], [16695...","[[16712.48, 0.015], [16712.49, 0.025], [16712...."
1,1671462568581,867920108,BTC_USD,"[[16695.21, 0.025], [16695.2, 0.2883], [16689....","[[16712.49, 0.025], [16712.5, 0.2883], [16721,..."
2,1671462569839,867920204,BTC_USD,"[[16701.09, 0.0002], [16695.22, 0.008], [16695...","[[16712.48, 0.0035], [16712.49, 0.025], [16712..."
3,1671462570761,867920301,BTC_USD,"[[16701.1, 0.025], [16701.09, 0.0002], [16695....","[[16712.49, 0.025], [16712.5, 0.2883], [16721,..."
4,1671462571780,867920423,BTC_USD,"[[16701.1, 0.025], [16701.09, 0.0002], [16695....","[[16712.48, 0.0003], [16712.49, 0.025], [16712..."


In [19]:
df2 = pd.DataFrame(df)

df2[[f'bid_{i}' for i in range(20)]] = pd.DataFrame(df2.bids.tolist(), index= df2.index)
df2[[f'ask_{i}' for i in range(20)]] = pd.DataFrame(df2.asks.tolist(), index= df2.index)

df2

,t,lastUpdateId,s,bids,asks,bid_0,bid_1,bid_2,bid_3,bid_4,...,ask_10,ask_11,ask_12,ask_13,ask_14,ask_15,ask_16,ask_17,ask_18,ask_19
0,1671462567378,867920009,BTC_USD,"[[16695.22, 0.0063], [16695.21, 0.025], [16695...","[[16712.48, 0.015], [16712.49, 0.025], [16712....","[16695.22, 0.0063]","[16695.21, 0.025]","[16695.2, 0.2883]","[16689.34, 0.0484]","[16686.8, 0.551]",...,"[16732.9, 0.005]","[16732.92, 0.005]","[16732.95, 0.005]","[16732.98, 0.005]","[16733, 0.005]","[16733.03, 0.005]","[16733.05, 0.005]","[16733.07, 0.005]","[16733.1, 0.005]","[16733.12, 0.005]"
1,1671462568581,867920108,BTC_USD,"[[16695.21, 0.025], [16695.2, 0.2883], [16689....","[[16712.49, 0.025], [16712.5, 0.2883], [16721,...","[16695.21, 0.025]","[16695.2, 0.2883]","[16689.34, 0.0484]","[16686.8, 0.551]","[16684.71, 0.0035]",...,"[16732.92, 0.005]","[16732.95, 0.005]","[16732.98, 0.005]","[16733, 0.005]","[16733.03, 0.005]","[16733.05, 0.005]","[16733.07, 0.005]","[16733.1, 0.005]","[16733.12, 0.005]","[16733.19, 0.005]"
2,1671462569839,867920204,BTC_USD,"[[16701.09, 0.0002], [16695.22, 0.008], [16695...","[[16712.48, 0.0035], [16712.49, 0.025], [16712...","[16701.09, 0.0002]","[16695.22, 0.008]","[16695.21, 0.025]","[16695.2, 0.2883]","[16689.34, 0.0484]",...,"[16732.9, 0.005]","[16732.92, 0.005]","[16732.95, 0.005]","[16732.98, 0.005]","[16733, 0.005]","[16733.03, 0.005]","[16733.05, 0.005]","[16733.07, 0.005]","[16733.1, 0.005]","[16733.12, 0.005]"
3,1671462570761,867920301,BTC_USD,"[[16701.1, 0.025], [16701.09, 0.0002], [16695....","[[16712.49, 0.025], [16712.5, 0.2883], [16721,...","[16701.1, 0.025]","[16701.09, 0.0002]","[16695.2, 0.2883]","[16689.34, 0.0484]","[16686.8, 0.551]",...,"[16732.92, 0.005]","[16732.95, 0.005]","[16732.98, 0.005]","[16733, 0.005]","[16733.03, 0.005]","[16733.05, 0.005]","[16733.07, 0.005]","[16733.1, 0.005]","[16733.12, 0.005]","[16733.19, 0.005]"
4,1671462571780,867920423,BTC_USD,"[[16701.1, 0.025], [16701.09, 0.0002], [16695....","[[16712.48, 0.0003], [16712.49, 0.025], [16712...","[16701.1, 0.025]","[16701.09, 0.0002]","[16695.2, 0.2883]","[16686.8, 0.551]","[16684.99, 0.0063]",...,"[16732.9, 0.005]","[16732.92, 0.005]","[16732.95, 0.005]","[16732.98, 0.005]","[16733, 0.005]","[16733.03, 0.005]","[16733.05, 0.005]","[16733.07, 0.005]","[16733.1, 0.005]","[16733.12, 0.005]"


In [20]:
for i in range(20):
    df2[[f'bid_{i}_px', f"bid_{i}_qty"]] = pd.DataFrame(df2[f"bid_{i}"].tolist(), index= df2.index).apply(pd.to_numeric, errors = 'coerce')
    df2[[f'ask_{i}_px', f"ask_{i}_qty"]] = pd.DataFrame(df2[f"ask_{i}"].tolist(), index= df2.index).apply(pd.to_numeric, errors = 'coerce')
    
#     df2[[f'bid_{i}_px', f"bid_{i}_qty"]] = df[[f'bid_{i}_px', f"bid_{i}_qty"]].apply(pd.to_numeric, errors = 'coerce')
#     df2[[f'ask_{i}_px', f"ask_{i}_qty"]] = df[[f'ask_{i}_px', f"ask_{i}_qty"]].apply(pd.to_numeric, errors = 'coerce')
# print(df2["bids"])
# print(df2["asks"])
df2

/Users/kritiksatija/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3636: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


,t,lastUpdateId,s,bids,asks,bid_0,bid_1,bid_2,bid_3,bid_4,...,ask_17_px,ask_17_qty,bid_18_px,bid_18_qty,ask_18_px,ask_18_qty,bid_19_px,bid_19_qty,ask_19_px,ask_19_qty
0,1671462567378,867920009,BTC_USD,"[[16695.22, 0.0063], [16695.21, 0.025], [16695...","[[16712.48, 0.015], [16712.49, 0.025], [16712....","[16695.22, 0.0063]","[16695.21, 0.025]","[16695.2, 0.2883]","[16689.34, 0.0484]","[16686.8, 0.551]",...,16733.07,0.005,16588.19,0.0024,16733.10,0.005,16582.92,0.0326,16733.12,0.005
1,1671462568581,867920108,BTC_USD,"[[16695.21, 0.025], [16695.2, 0.2883], [16689....","[[16712.49, 0.025], [16712.5, 0.2883], [16721,...","[16695.21, 0.025]","[16695.2, 0.2883]","[16689.34, 0.0484]","[16686.8, 0.551]","[16684.71, 0.0035]",...,16733.10,0.005,16588.19,0.0024,16733.12,0.005,16582.92,0.0326,16733.19,0.005
2,1671462569839,867920204,BTC_USD,"[[16701.09, 0.0002], [16695.22, 0.008], [16695...","[[16712.48, 0.0035], [16712.49, 0.025], [16712...","[16701.09, 0.0002]","[16695.22, 0.008]","[16695.21, 0.025]","[16695.2, 0.2883]","[16689.34, 0.0484]",...,16733.07,0.005,16596.20,0.0159,16733.10,0.005,16588.19,0.0024,16733.12,0.005
3,1671462570761,867920301,BTC_USD,"[[16701.1, 0.025], [16701.09, 0.0002], [16695....","[[16712.49, 0.025], [16712.5, 0.2883], [16721,...","[16701.1, 0.025]","[16701.09, 0.0002]","[16695.2, 0.2883]","[16689.34, 0.0484]","[16686.8, 0.551]",...,16733.10,0.005,16582.92,0.0326,16733.12,0.005,16580.22,0.0030,16733.19,0.005
4,1671462571780,867920423,BTC_USD,"[[16701.1, 0.025], [16701.09, 0.0002], [16695....","[[16712.48, 0.0003], [16712.49, 0.025], [16712...","[16701.1, 0.025]","[16701.09, 0.0002]","[16695.2, 0.2883]","[16686.8, 0.551]","[16684.99, 0.0063]",...,16733.07,0.005,16588.19,0.0024,16733.10,0.005,16582.92,0.0326,16733.12,0.005


<h2>Final dataframe format</h2>

In [21]:
#Remove unnecessary columns
df2.drop(columns = [f'bid_{i}' for i in range(20)]+[f'ask_{i}' for i in range(20)]+["bids","asks", "lastUpdateId"], axis=1, inplace = True)

df2

,t,s,bid_0_px,bid_0_qty,ask_0_px,ask_0_qty,bid_1_px,bid_1_qty,ask_1_px,ask_1_qty,...,ask_17_px,ask_17_qty,bid_18_px,bid_18_qty,ask_18_px,ask_18_qty,bid_19_px,bid_19_qty,ask_19_px,ask_19_qty
0,1671462567378,BTC_USD,16695.22,0.0063,16712.48,0.0150,16695.21,0.0250,16712.49,0.0250,...,16733.07,0.005,16588.19,0.0024,16733.10,0.005,16582.92,0.0326,16733.12,0.005
1,1671462568581,BTC_USD,16695.21,0.0250,16712.49,0.0250,16695.20,0.2883,16712.50,0.2883,...,16733.10,0.005,16588.19,0.0024,16733.12,0.005,16582.92,0.0326,16733.19,0.005
2,1671462569839,BTC_USD,16701.09,0.0002,16712.48,0.0035,16695.22,0.0080,16712.49,0.0250,...,16733.07,0.005,16596.20,0.0159,16733.10,0.005,16588.19,0.0024,16733.12,0.005
3,1671462570761,BTC_USD,16701.10,0.0250,16712.49,0.0250,16701.09,0.0002,16712.50,0.2883,...,16733.10,0.005,16582.92,0.0326,16733.12,0.005,16580.22,0.0030,16733.19,0.005
4,1671462571780,BTC_USD,16701.10,0.0250,16712.48,0.0003,16701.09,0.0002,16712.49,0.0250,...,16733.07,0.005,16588.19,0.0024,16733.10,0.005,16582.92,0.0326,16733.12,0.005


<h2>Placing market orders</h2>

In [22]:
#Assumption:
#Orders beyond the liquidity of the orderbook
#will be executed at the worst price
def place_market_order(side = "buy", dollar_value = 10000):
    
    price = []

    for row in df2.to_dict(orient="records"):
        
        side = "ask" if side == "buy" else "bid"
            
        total_price = 0
        total_qty = 0

        for i in range(20):
            if(total_price < dollar_value):
                total_price += row[f"{side}_{i}_px"] * row[f"{side}_{i}_qty"]
                total_qty += row[f"{side}_{i}_qty"]
                last = str(i)
                
#         print(f"Total {order_type}: {total_price}")
        
        if total_price >= dollar_value:
            extra = total_price - dollar_value
            total_qty = total_qty - extra / row[f"{side}_{last}_px"]
        else:
            extra = dollar_value - total_price
            total_qty = total_qty + extra / row[f"{side}_{last}_px"]

#         print(total_price)
        
        avg_price = dollar_value / total_qty
        price.append(avg_price)
    print(price)

In [23]:
place_market_order("sell", 2000)

[16695.203138698194, 16695.20208690026, 16695.213255999322, 16696.44119674212, 16696.44119674212]


In [24]:
# from functools import partial

# best_bid = -1
# limit_orders = []

# # define your callback function on message received
# def check(conn: Connection, response: WebSocketResponse, x):
#     global best_bid
#     global orders
    
#     print(x)
    
#     if response.error:
#         print('error returned: ', response.error)
#         conn.close()
#         return
    
#     data = response.result
    
#     if list(data.keys())[0] != "status":
#         best_bid_local = data['bids'][0][0]
#         print(best_bid)
#         best_bid = best_bid_localbb
        
        
# async def order_stream():
#     channel_name = "BTC_USD"
#     depth = "5"
#     update_rate = "1000ms"
    
#     add_part = partial(check, x = 5)
    
#     conn = Connection(Configuration())

#     channel = SpotOrderBookChannel(conn, add_part)
    
#     channel.subscribe([channel_name, depth, update_rate])
    
#     await conn.run()
    
     

# start_time = time.perf_counter()
# await order_stream()
# end_time = time.perf_counter()
# print(f"Time: {end_time - start_time}")

In [25]:
best_bid = -1
limit_orders = []

def check(conn: Connection, response: WebSocketResponse):
    global best_bid
    global limit_orders
#     print(limit_orders)
    side = limit_orders[0][0]
    price = float(limit_orders[0][1])
    qty = float(limit_orders[0][2])
    
    if response.error:
        print('error returned: ', response.error)
        conn.close()
        return
    
    data = response.result
    
#     print(data)
#     time.sleep(1000)
    
    if list(data.keys())[0] != "status":
#         print(data)
#         best_local = float(data[f"{side}"][0][0])
        best_local = float(data[f"{side}"])
        print(f"order_price: {price}, best_{side}_local = {best_local}")
        if side == "b":
            if price < best_local:
                print(f"Order executed at price: {best_local}")
                conn.close()
                return
        if side == "a":
            if price > best_local:
                print(f"Order executed at price: {best_local}")
                conn.close()
                return
        
        
async def order_stream():
    channel_name = "BTC_USD"
    depth = "5"
    update_rate = "1000ms"
    
    conn = Connection(Configuration())

#     channel = SpotOrderBookChannel(conn, check)
    channel = SpotBookTickerChannel(conn, check)
    
#     channel.subscribe([channel_name, depth, update_rate])
    channel.subscribe([channel_name])
    
    await conn.run()

In [26]:
#Assumption
#Once best bid / ask price crosses the limit price, we assume the entire quantity is executed
async def place_limit_order(side = "sell", price = 20000, qty = 1):
    side = "a" if side == "buy" else "b"
    limit_orders.append([side, price, qty])
    await order_stream()
#     await check()

In [27]:
await place_limit_order("bid", 16694, 1)

order_price: 16694.0, best_b_local = 16705.21
Order executed at price: 16705.21
